# Ant Colony Optimization

## Import ed iperparametri

In [17]:
import numpy as np
from numpy.random import choice as np_choice

In [18]:
# Matrice delle distanze
distances = np.array([[np.inf, 2, 2, 5, 7],
                      [2, np.inf, 4, 8, 2],
                      [2, 4, np.inf, 1, 3],
                      [5, 8, 1, np.inf, 2],
                      [7, 2, 3, 2, np.inf]])
n_ants = 1 # numero di agenti per iterazione
# Numero di agenti con le performance migliori
n_best = 1
n_iter = 100 # numero di iterazioni
decay = 0.95 # fattore di evaporazione
alfa = 1 # parametro alfa del feromone
beta = 1 # parametro beta della distanza

## Classe della colonia di formiche

In [19]:
class AntColony(object):

    # COSTRUTTORE
    def __init__(self, distances, n_ants, n_best, n_iter, decay, alfa, beta):
        self.distances = distances
        # Si inizializza la matrice dei feromoni con
        # una matrice di tutti 1 diviso la lunghezza
        # del vettore delle distanze
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        # Lista degli indici di ogni nodo
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iter = n_iter
        self.decay = decay
        self.alfa = alfa
        self.beta = beta

    # Scegli la prossima citta'
    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        # Azzera i feromoni visitati
        pheromone[list(visited)] = 0
        # Calcola l'attrattivita' normalizzata di ogni citta'
        row = pheromone ** self.alfa * ((1.0/ dist) ** self.beta)
        norm_row = row / row.sum()
        # Scelta aleatoria della prossima città
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move

    # Genera il percorso
    def gen_path(self, start):
        path = [] # percorso
        visited = set()
        visited.add(start)
        prev = start
        for ind in range(len(self.distances)-1):
            # Muoviti
            move = self.pick_move(self.pheromone[prev],
                                  self.distances[prev],
                                  visited)
            # Aggiungi il precedente e il nuovo
            path.append((prev, move))
            # Aggiorna il precedente
            prev = move
            # Aggiungi il nuovo ai visitati
            visited.add(move)
        # Torna al punto di partenza
        path.append((prev, start))
        return path

    # Calcola di distanza di un percorso
    def gen_path_dist(self, path):
        dist = 0
        for elem in path:
            dist += self.distances[elem]
        return dist

    # Calcola di distanza di ogni percorso
    def gen_all_paths(self):
        all_paths = []
        for ind in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths
    
    # Aggiorna il feromone
    def spread_pheromone(self, all_paths, n_best, shortest_path):
        # Ordina i percorsi in base alla distanza
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        # Considera solo i percorsi migliori
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                # Aumenta il feromone in ogni percorso
                self.pheromone[move] += 1.0 / self.distances[move]

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for ind in range(self.n_iter):
            all_paths = self.gen_all_paths()
            self.spread_pheromone(all_paths,
                                  self.n_best, 
                                  shortest_path = shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < all_time_shortest_path[1]:
                # Trovato percorso migliore
                all_time_shortest_path = shortest_path
            # Evoporazione
            self.pheromone *= self.decay
        return all_time_shortest_path

## Main

In [ ]:
ant_colony = AntColony(distances, n_ants, n_best, n_iter, decay, alfa, beta)
best_path, best_dist = ant_colony.run()
best_path_clean = [(int(a), int(b)) for a, b in best_path]
print("shortest_path:", best_path_clean, "distanza:", float(best_dist))

shortest_path: [(0, 2), (2, 3), (3, 4), (4, 1), (1, 0)] distanza: 9.0
